# Intended audience for this notebook

I wrote this notebook after I completed watching lesson 1 & 2, and reading about datablock and dataset API in the links mentioned below
1. https://asvcode.github.io/Blogs/fastai/augmentation/image-augmentation/2020/03/26/Fastai2-Image-Augmentation.html
2. https://towardsdatascience.com/advanced-dataloaders-with-fastai2-ecea62a7d97e
3. https://towardsdatascience.com/working-with-3d-data-fastai2-5e2baa09037e

So this notebook is geared towards beginners of fastai API

# Strategy

Based on the first two lessons, the steps to create a `CNN` classification learner are
1. create dataloader using either 
    - `ImageDataLoaders` or
    - `DataBlock`
2. feed the dataloader to create a learner using `cnn_learner`
3. fine-tune the learner
3. use `ClassificationInterpretation` to plot confusion matrix and top losses to understand performance of each label
4. use `ImageClassifierCleaner` to clean images (This step may not be relevant to this challenge)

# step 0

In [ ]:
import fastai
import shutil

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
from fastai.vision.all import *
from fastai.vision.core import *

InteractiveShell.ast_node_interactivity = "all"

train = False

## add data sources

Add `~/.torch` location to copy existing architectures to run the model

In [ ]:
model_loc = Path('/root/.cache/torch/hub/checkpoints/')
if not model_loc.exists():
    model_loc.mkdir(exist_ok=True, parents=True)
model_loc.ls()

### existing architectures

In [ ]:

Path("../input/cassava-leaf-disease-classification/").exists() == Path("/root/.cache/torch/hub/checkpoints/").exists()
shutil.copy(f"{Path(os.getcwd()).parent}/input/cassava-modelresnet34fine-tune10pkl/resnet34-333f7ec4.pth", "/root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth")
shutil.copy(f"{Path(os.getcwd()).parent}/input/cassava-modelresnet34fine-tune10pkl/resnet50-19c8e357.pth", "/root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth")

# step 1 - create dataloader

In [ ]:
fastai.__version__

In [ ]:
path = Path('../input/cassava-leaf-disease-classification')
os.listdir(path)

In [ ]:
labels = pd.read_csv(path / 'train.csv')
labels["image_id"] = labels["image_id"].apply(
    lambda x: f'train_images/{x}')
labels.head()

## using ImageDataloader

I used `ImageDataLoaders` to create dataloaders but datablocks or datasets api offers lot of flexibility in creating dataloaders

In [ ]:
dls = ImageDataLoaders.from_df(
    labels, 
    path='../input/cassava-leaf-disease-classification/',
    bs=64,
    fn_col=0, 
    label_col=1, 
    seed=42,
    valid_pct=0.2,
    item_tfms=RandomResizedCrop(512, min_scale=0.75, ratio=(1.,1.)),
    batch_tfms=aug_transforms(),
    num_workers=0)

dls.valid_ds.items[:3]

# step 2 - create learner

`fp16` conversion can help reduce the gpu memory requirement and may slightly reduce the accuracy. But it offers an option to change the architecture from resnet34(until version 6) to resnet50 along with decent batch size. If you get any memory errors reduce the batch size (`bs`) or change the architecture 

`mixup` helps in regularizing the model as it is forced to learn not just a single label from an image but a combination of labels

These concepts (`mixup` and `fp16`) are not discussed in lesson 1 or 2. I haven't used these concepts until version 6 of this notebook. So if you want barebones fastai implementation, have a look at version 6 or earlier version of this notebook

In [ ]:
learn = cnn_learner(dls, resnet50, metrics=[error_rate, accuracy]).to_native_fp16()

# step 3 - fine tune the learner

`Mixup` is added here instead of as option to `cnn_learner`. This is to restrict use of `Mixup` to learner training  

In [ ]:
if train:
    learn.fine_tune(10, cbs=[MixUp(0.5)])

# step 4 - save model

`cnn_learner` is converted to `fp32` so that there won't be issues during validation. I am entirely new to `fp16` training. This concept is not mentioned in lesson 1 or 2

In [ ]:
model_name = "resnet50-fine_tune-10_resize-512"

if train:
    learn.export(Path(f"{Path(os.getcwd()).parent}/output/cassava-leaf-disease-classification/{model_name}.pkl"))
else:
    # The learner should be loaded on gpu to enable test time augumentation. Thanks to @muellerzr for the tip
    learn = load_learner(Path(f"{Path(os.getcwd()).parent}/input/cassava-modelresnet34fine-tune10pkl/{model_name}.pkl"), cpu=False)
    
learn = learn.to_native_fp32()
learn

# step 5 - plot confusion matrix

In [ ]:
if train:
    interp = ClassificationInterpretation.from_learner(learn)
    interp.plot_confusion_matrix()

## Cleaner

In [ ]:
if train:
    from fastai.vision.widgets import *
    cleaner = ImageClassifierCleaner(learn)
    cleaner

# submission

Test dataloader is creating using the instructions mentioned [here]( https://forums.fast.ai/t/a-brief-guide-to-test-sets-in-v2-you-can-do-labelled-now-too/57054)

In [ ]:
test_labels = pd.read_csv(path / 'sample_submission.csv')
test_labels["image_id"] = test_labels["image_id"].apply(
    lambda x: f'test_images/{x}')

test_labels.head()

In [ ]:
test_predictions = []

for i, r in test_labels.iterrows():
    test_predictions.append(int(learn.predict(f'../input/cassava-leaf-disease-classification/{r.image_id}')[0]))
    
test_predictions[0]

In [ ]:
# test_dl = dls.test_dl(test_labels, with_labels=True)
# test_dl.show_batch()
# test_dl.items

# preds = learn.get_preds(dl=test_dl)
# # preds = learn.tta(dl=test_dl, n=8, beta=0) # I learnt this trick from fastai forums
# test_predictions = preds[1]
# test_predictions[0]

In [ ]:
submission = pd.read_csv(path / 'sample_submission.csv')
submission['label'] = test_predictions
submission.head()
submission.to_csv('submission.csv',index=False)